# 🎯 使用 GitHub 模型进行规划与设计模式 (Python)

## 📋 学习目标

本笔记本展示了使用 Microsoft Agent Framework 和 GitHub 模型构建智能代理的高级规划与设计模式。您将学习如何创建能够分解复杂问题、规划多步骤解决方案并协调复杂工作流程的代理。

**您将掌握的高级能力：**
- 🧠 **战略规划**：将复杂任务分解为可管理的子任务
- 🗺️ **多步骤推理**：顺序和并行任务执行模式
- 🎯 **目标导向设计**：代理专注于实现特定目标
- 🔄 **自适应规划**：根据上下文动态调整策略

## 🎯 规划架构概念

### 核心规划组件
- **任务分解**：将复杂问题分解为更小、更易管理的部分
- **执行规划**：确定子任务的最佳顺序和依赖关系
- **资源管理**：高效分配工具和能力
- **进度监控**：跟踪完成情况并适应变化

### 规划设计模式
- **策略模式**：针对不同场景的多种规划方法
- **责任链模式**：顺序处理并提供备用选项
- **命令模式**：封装任务执行，支持撤销/重做功能
- **观察者模式**：进度跟踪和事件驱动更新

## 🏗️ 技术架构

### 规划系统组件
- **Microsoft Agent Framework**：支持高级规划的 Python 实现
- **GitHub 模型集成**：高性能推理与决策支持
- **任务编排**：协调复杂工作流程的执行  
- **状态管理**：持续跟踪规划进度和结果

### 规划流程图
```python
Complex Goal → Task Analysis → Subtask Decomposition → Execution Planning
                ↓                    ↓                    ↓
         Priority Assessment → Resource Allocation → Sequential Execution
                ↓                    ↓                    ↓
         Progress Monitoring → Adaptive Replanning → Goal Achievement
```

## 🧠 规划方法论

### 1. **层次化任务规划**
- 自上而下分解复杂目标
- 带有依赖关系的嵌套目标结构
- 对子目标进行递归规划
- 跨层级高效分配资源

### 2. **顺序规划**
- 按步骤执行任务，明确依赖关系
- 每阶段的错误处理与恢复
- 进度检查点与验证
- 对失败步骤的回滚能力

### 3. **并行规划**
- 独立任务的并发执行
- 资源同步与冲突解决
- 通过并行化优化性能
- 协调完成与结果汇总

## ⚙️ 前提条件与设置

**所需依赖项：**
```bash

pip install agent-framework-core -U
```

**环境配置 (.env 文件)：**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

## 🎨 规划设计模式

### 目标导向规划
- **SMART 目标**：具体、可衡量、可实现、相关、时间限定的目标
- **里程碑跟踪**：进度测量与验证检查点
- **成功标准**：明确任务完成的定义
- **自适应策略**：根据反馈动态调整规划

### 资源感知规划
- **能力评估**：了解可用工具及其限制
- **负载均衡**：在可用资源间优化任务分配
- **约束管理**：在系统和 API 限制内工作
- **性能优化**：最大化效率并最小化延迟

### 错误弹性规划
- **故障检测**：早期识别规划或执行问题
- **恢复策略**：自动回退与替代方法
- **状态保留**：在错误恢复期间保持进度
- **优雅降级**：在无法完全完成时实现部分成功

## 🚀 高级规划功能

- **元规划**：能够规划如何规划的代理
- **协作规划**：多代理协调处理复杂项目
- **从经验中学习**：随着时间推移改进规划策略
- **实时适应**：根据变化的条件动态重新规划

## 📊 用例与应用

### 业务流程自动化
- 项目管理与任务调度
- 工作流程优化与资源分配
- 战略规划与决策支持
- 流程改进与自动化

### 研究与分析
- 文献综述与综合
- 数据分析管道规划
- 实验设计与执行
- 报告生成与格式化

### 创意项目
- 内容创作工作流程
- 多媒体项目协调
- 活动策划与执行
- 活动组织与管理

准备好构建能够解决复杂多步骤挑战的智能规划代理了吗？让我们设计一些复杂问题的解决能力吧！🧠✨


In [1]:

! pip install agent-framework-core -U 

In [2]:
# 📦 Import Standard Libraries
# os: For environment variable access
# dotenv: For loading environment variables from .env file
import os

from dotenv import load_dotenv

In [3]:
# 📝 Import Data Modeling Libraries
# Pydantic: For data validation and structured outputs
# BaseModel: Base class for creating data models
# Field: For adding metadata and validation to model fields
# List: Type hint for list collections
from pydantic import BaseModel, Field
from typing import List

In [4]:
# 🤖 Import Microsoft Agent Framework Components
# ChatMessage: For creating structured chat messages
# Role: Enum for message roles (user, assistant, system)
# ChatOptions: Configuration options for chat interactions
# OpenAIChatClient: Client for OpenAI-compatible APIs (GitHub Models)
from agent_framework import ChatMessage, Role, ChatOptions
from agent_framework.openai import OpenAIChatClient

In [5]:
# 🔧 Load Environment Variables
# Load configuration from .env file
# Required: GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL_ID
load_dotenv()

True

In [6]:
# 🎯 Agent Configuration
# Define the agent's name and role

AGENT_NAME = "TravelAgent"

# 📋 System Instructions for Planning Agent
# This agent acts as a coordinator that decides which specialized agents to use
AGENT_INSTRUCTIONS = """You are a planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialized in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
"""


In [7]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [8]:
options = ChatOptions(response_format=TravelPlan)

In [9]:
client = OpenAIChatClient(base_url=os.environ.get("GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id=os.environ.get("GITHUB_MODEL_ID"))

In [10]:
agent =  client.create_agent(name= AGENT_NAME , instructions=AGENT_INSTRUCTIONS)

In [11]:
messages = [
        ChatMessage(role=Role.USER, text="Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne")
    ]

In [12]:
response = await agent.run(messages,response_format=TravelPlan)

In [13]:
response.messages[0].text

'{"main_task":"Plan a family trip from Singapore to Melbourne for 4 people, including 2 kids.","subtasks":[{"assigned_agent":"FlightBooking","task_details":"Book one round-trip flight from Singapore to Melbourne for a family of 4, considering family-friendly options."},{"assigned_agent":"HotelBooking","task_details":"Find and book a family-friendly hotel in Melbourne that can accommodate 4 people, including 2 kids."},{"assigned_agent":"CarRental","task_details":"Arrange a car rental in Melbourne for the family to facilitate travel around the city."},{"assigned_agent":"ActivitiesBooking","task_details":"Provide a list of kid-friendly activities and attractions in Melbourne for the family to enjoy."}]}'


---

**免责声明**：  
本文档使用AI翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
